In [2]:
import boto3
import pandas as pd
from sentence_transformers import SentenceTransformer
from io import StringIO
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import sagemaker
import os
import joblib
from fuzzy import fmcc
import code_variables

### Save model

In [3]:
if __name__ == "__main__":
    cc_object = fmcc()
    joblib.dump(cc_object, f"{code_variables.model_name}.pkl")

## Endpoint creation

### Upload the pre-trained model mode.tar.gz file to S3

In [4]:
model_file_name = f"{code_variables.model_name}.pkl"
prefix = f"model_tars/{code_variables.model_name}/"
!tar -czf fm_catalog.tar.gz $model_file_name 

In [5]:
prefix=''
fObj = open("fm_catalog.tar.gz", "rb") 
key = os.path.join(prefix, "fm_catalog.tar.gz") 
bucket = code_variables.bucket_name
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj) 

### Create SageMaker model endpoint with python SDK

In [6]:
model_data = "s3://{}/{}".format(bucket, key) 
print(f"model data: {model_data}") 

model data: s3://aiops.d-aiml-315952988300-us-east-2/fm_catalog.tar.gz


In [7]:
inference_file_path = "inference.py"
requirements_file_path = "requirements.txt"
variable_file = "code_variables.py"
model_file='fuzzy.py'
prefix = "model_tars/fm_catalog/"

In [8]:
!tar -cvzf sourcedir.tar.gz $inference_file_path $requirements_file_path $model_file $variable_file

inference.py
requirements.txt
fuzzy.py
code_variables.py


In [9]:
fObj = open("sourcedir.tar.gz", "rb")
key = os.path.join(prefix, "sourcedir.tar.gz")
bucket = "aiops.d-aiml-315952988300-us-east-2"
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

In [10]:
sourcedir_data = "s3://{}/{}".format(bucket, key)
print(f"model data: {sourcedir_data}")

model data: s3://aiops.d-aiml-315952988300-us-east-2/model_tars/fm_catalog/sourcedir.tar.gz


### Sagemaker Model Deployment

In [11]:
sagemaker_client = boto3.client('sagemaker')
role = sagemaker.get_execution_role()
try:
    sagemaker_client.delete_model(ModelName = 'fm-catalog')
except Exception as e:
    print('No exisiting endpoint')
    print(e)

### Deploy with Python SDK

In [12]:
sagemaker_client.create_model( 
        ModelName='fm-catalog', 
        PrimaryContainer={ 
            'Image': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3', 
            'ModelDataUrl': 's3://aiops.d-aiml-315952988300-us-east-2/fm_catalog.tar.gz', 
            'Environment': { 
                'SAGEMAKER_CONTAINER_LOG_LEVEL': '20', 
                'SAGEMAKER_PROGRAM': 'inference.py', 
                'SAGEMAKER_REGION': 'us-east-2', 
                'SAGEMAKER_SUBMIT_DIRECTORY': 's3://aiops.d-aiml-315952988300-us-east-2/model_tars/fm_catalog/sourcedir.tar.gz'}}, 
        ExecutionRoleArn= role
) 

{'ModelArn': 'arn:aws:sagemaker:us-east-2:315952988300:model/fm-catalog',
 'ResponseMetadata': {'RequestId': 'e329f5d3-b98f-4fa0-911a-2f467eef106d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e329f5d3-b98f-4fa0-911a-2f467eef106d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '72',
   'date': 'Thu, 01 Jun 2023 15:21:47 GMT'},
  'RetryAttempts': 0}}

In [14]:
try:
    sagemaker_client.delete_endpoint_config(EndpointConfigName = 'fm-catalog-v1')
except Exception as e:
    print('No existing config')
    print(e)
sagemaker_client.create_endpoint_config( 
    EndpointConfigName='fm-catalog-v1', 
    ProductionVariants=[ 
        { 
            'VariantName': 'AllTraffic', 
            'ModelName': 'fm-catalog', 
            'InitialInstanceCount': 1, 
            'InstanceType': 'ml.m5.large' 
        }, 
    ] 
) 


{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:315952988300:endpoint-config/fm-catalog-v1',
 'ResponseMetadata': {'RequestId': '168ffd9d-d903-4fae-acd9-9f0327f513dc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '168ffd9d-d903-4fae-acd9-9f0327f513dc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Thu, 01 Jun 2023 15:22:47 GMT'},
  'RetryAttempts': 0}}

In [15]:
try:
    sagemaker_client.delete_endpoint(EndpointName = 'fm-catalog-v1')
except Exception as e:
    print('No exisiting endpoint')
    print(e)

In [16]:
sagemaker_client.create_endpoint( 
    EndpointName='fm-catalog-v1', 
    EndpointConfigName='fm-catalog-v1', 
) 

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:315952988300:endpoint/fm-catalog-v1',
 'ResponseMetadata': {'RequestId': '545a5474-dd6b-481b-bf8d-0c66b3ba6d90',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '545a5474-dd6b-481b-bf8d-0c66b3ba6d90',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '81',
   'date': 'Thu, 01 Jun 2023 15:22:50 GMT'},
  'RetryAttempts': 0}}